In [1]:
!nvidia-smi

Wed May 18 05:49:09 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
import sys
drive.mount('/content/drive')
#设置路径
sys.path.append('/content/drive/MyDrive/Colab Notebooks')

Mounted at /content/drive


In [3]:
! pip install -U synonyms

     |████████████████████████████████| 10.7 MB 904 kB/s 
  Created wheel for synonyms: filename=synonyms-3.16.0-py3-none-any.whl size=10832785 sha256=434e547356625062a5d7f138e22719ecae52edcc76b3c977a1656e401353bace
  Stored in directory: /root/.cache/pip/wheels/e4/cd/43/b4548753509a94471fc946967a07116252d49aeeb689db8f7c
Successfully built synonyms


In [4]:
! pip install torch==1.6.0

     |████████████████████████████████| 748.8 MB 18 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.6.0 which is incompatible.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.6.0 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.6.0 which is incompatible.


In [5]:
! pip install transformers==4.0.1

     |████████████████████████████████| 1.4 MB 9.7 MB/s 
     |████████████████████████████████| 880 kB 39.3 MB/s 
     |████████████████████████████████| 2.9 MB 52.7 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=adba1b2374be2e04ed8f2450a5ac36cdd355079e08837f5c5ab55b8468c3e5a8
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [6]:
import synonyms

[jieba] default dict file path ../data/vocab.txt
[jieba] default dict file path ../data/vocab.txt
[jieba] load default dict ../data/vocab.txt ...
[jieba] load default dict ../data/vocab.txt ...
>> Synonyms load wordseg dict [/usr/local/lib/python3.7/dist-packages/synonyms/data/vocab.txt] ... 
>> Synonyms on loading stopwords [/usr/local/lib/python3.7/dist-packages/synonyms/data/stopwords.txt] ...
[Synonyms] on loading vectors [/usr/local/lib/python3.7/dist-packages/synonyms/data/words.vector.gz] ...

[Synonyms] downloading data from https://github.com/chatopera/Synonyms/releases/download/3.15.0/words.vector.gz to /usr/local/lib/python3.7/dist-packages/synonyms/data/words.vector.gz ... 
 this only happens if SYNONYMS_WORD2VEC_BIN_URL_ZH_CN is not present and Synonyms initialization for the first time. 
 It would take minutes that depends on network.

[Synonyms] downloaded.



## EDA（Easy Data Augmentation）
![EDA3](https://img-blog.csdnimg.cn/50c22b4212714b509ce053ff921d6bdd.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

### 对于训练集中的给定句子，随机选择并执行以下操作之一：
* 同义词替换（SR）：从句子中随机选择 n 个不是停用词的词。 用随机选择的同义词之一替换这些单词中的每一个。
* 随机插入 (RI)：在句子中随机找到一个词，并找出其同义词，且该同义词不是停用词。 将该同义词插入句子中的随机位置。 这样做n次。
* 随机交换（RS）：随机选择句子中的两个单词并交换它们的位置。 这样做n次。
* 随机删除（RD）：以概率 p 随机删除句子中的每个单词。

In [7]:
# 读取停用词表
import random
import re
from random import shuffle
# strip() 方法用于移除字符串头尾指定的字符（默认为空格或换行符）或字符序列。
stop_words = {word.strip() for word in open('/content/drive/MyDrive/Colab Notebooks/dataset/baidu_stopwords.txt', 'r', encoding='utf8').readlines()}

In [8]:
# 同义词替换（SR）：从句子中随机选择 n 个不是停用词的词。 用随机选择的同义词之一替换这些单词中的每一个。

# 得到同义词列表
def get_synonym(word):
  # nearby返回一个元组，位置0返回同义词，位置1返回相似度
  sys = set(synonyms.nearby(word)[0])
  # 去除原词
  if word in sys:
    sys.remove(word)
  return list(sys)
  # 如果输入"给力" 可能没有同义词（同义词只有他自己） 则返回  ([],[])

# 同义词替换
# 传入一个词列表和替换词的数量
def synonym_replacement(words, n):

  new_words = words.copy()
  # 去除停用词，去重，变成列表
  random_word_list = list(set([word for word in words if word not in stop_words]))
  # 打乱
  random.shuffle(random_word_list)

  num_replaced = 0
  for random_word in random_word_list:
    synonym_words = get_synonym(random_word)
    if len(synonym_words)>=1:
        # 随机选取一个同义词
        synonym = random.choice(list(synonym_words))
        # 如果word是要替换的词 替换成同义词 ，否则返回原词
        new_words = [synonym if word == random_word else word for word in new_words]
        # 同义词替换数量+1
        num_replaced += 1
        
    if num_replaced >= n:
        break
  # 为什么加这两句话？ 看note1
  sentence = ' '.join(new_words)
  new_words = sentence.split(' ')

  return new_words

In [9]:
# 使用eda进行数据增强
# 同义词替换的比例、 增强的句子数目
def eda(sentence, alpha_sr=0.1, num_aug=9):

  # 位置0是词组、1是词性
  words = synonyms.seg(sentence)[0]
  num_words = len(words)
  n_sr = max(1, int(alpha_sr * num_words))

  augmented_sentences = []

  for _ in range(num_aug):
    a_words = synonym_replacement(words, n_sr)
    augmented_sentences.append(' '.join(a_words))
  return augmented_sentences

In [10]:
# note1
# 存在这样一种情况
sentence = ['in', 'war', 'one', 'must', 'be', 'a', 'good', 'actor']
word = 'actor'
example_synonyms = ['actress', 'film star', 'performer', 'comedian', 'entertainer']
new_sentence = ['in', 'war', 'one', 'must', 'be', 'a', 'good', 'film star']
# 为了消除空格
new_sentence = ['in', 'war', 'one', 'must', 'be', 'a', 'good', 'film', 'star']

In [11]:
eda('9月15日以来，台积电、高通、三星等华为的重要合作伙伴，只要没有美国的相关许可证，都无法供应芯片给华为，而中芯国际等国产芯片企业，也因采用美国技术，而无法供货给华为。目前华为部分型号的手机产品出现货少的现象，若该形势持续下去，华为手机业务将遭受重创。')

['9 月 15 日 以来 ， 台积电 、 高通 、 三星 等 金立 的 重要 合作伙伴 ， 只要 没有 美国 的 相关 许可权 ， 都 无法 供应 芯片 给 金立 ， 而 中芯国际 等 第五代 芯片 企业 ， 也 因 采用 美国 技术 ， 而 无法 交货 给 金立 。 目前 金立 部分 制式 的 手机 产品设计 出现 货 少 的 现象 ， 若 该 形势 持续 下去 ， 金立 手机 业务发展 将 遭受 重创 。',
 '9 月 15 日 以来 ， 台积电 、 ARM 、 三星 等 惠普 的 重要 合作伙伴 ， 只要 没有 新西兰 的 相关 许可证 ， 都 无法 配给 芯片 给 惠普 ， 而 中芯国际 等 国产 芯片 外贸企业 ， 也 因 采用 新西兰 技术 ， 而 无法 供货 给 惠普 。 目前 惠普 部分 型号 的 手机 产品 出现 货 相对来说 的 现象 ， 若 该 形势 持续 下去 ， 惠普 手机 业务 将 遭受 受创 。',
 '9 月 15 日 以来 ， 台积电 、 高通 、 三星 等 华为 的 重要 伙伴 ， 只要 没有 美国 的 相关 许可证 ， 都 无法 供应 芯片 给 华为 ， 而 中芯国际 等 国产 芯片 金融机构 ， 也 因 换用 美国 技术 ， 而 无法 供货 给 华为 。 目前 华为 部分 型号 的 手机 商品 出现 货 相对来说 的 现象 ， 若 该 形势 稳步 下去 ， 华为 手机 业务 将 遭受 挫败 。',
 '9 月初 15 翌年 以来 ， 三星电子 、 高通 、 三星 等 华为 的 重要 合作伙伴 ， 只要 没有 英国 的 相关 许可证 ， 都 无法 供应 显示卡 给 华为 ， 而 中芯国际 等 国产 显示卡 企业 ， 也 因 采用 英国 技术 ， 而 无法 供货 给 华为 。 目前 华为 部分 型号 的 笔记本电脑 产品 出现 货 少 的 现象 ， 若 该 形势 加速 下去 ， 华为 笔记本电脑 业务 将 遭受 重创 。',
 '9 月 15 日 以来 ， VIA 、 高通 、 三星 等 华为 的 重要 供应商 ， 只要 没有 美国 的 相关 许可证 ， 就 无法 供应 芯片 给 华为 ， 而 中芯国际 等 第五代 芯片 民营企业 ， 也 因 采用 美国 技术 ， 而 无法 供货商 给 华为 。 目前 华为 部分 型号 的 手

![UDA5](https://img-blog.csdnimg.cn/9d10da70d1d0467e93ef5bb1267ac87f.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)
**$ \tilde{\theta} $： 参数不变 不进行反向传播**


![在这里插入图片描述](https://img-blog.csdnimg.cn/88a3abe95bbd4e369fe4d085533c9c35.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

In [12]:
from bucket_sampler import SortedSampler, BucketBatchSampler
from EMA import *

In [13]:
import random
import numpy as np
import torch
config = {
        'train_file_path': '/content/drive/MyDrive/Colab Notebooks/dataset/ESIM/train.json',
        'dev_file_path': '/content/drive/MyDrive/Colab Notebooks/dataset/ESIM/dev.json',
        'test_file_path': '/content/drive/MyDrive/Colab Notebooks/dataset/ESIM/test.json',
        'output_path': '.',
        'model_path': '/content/drive/MyDrive/Colab Notebooks/dataset/BERT_model',
        'batch_size': 16,
        'num_epochs': 1,
        'max_seq_len': 64,
        'learning_rate': 2e-5,
        'weight_decay': 0.01,
        'use_bucket': True,
        'bucket_multiplier': 200,
        'unsup_data_ratio': 1.5,
        'uda_softmax_temp': 0.4,
        'uda_confidence_threshold': 0.8,
        'device': 'cuda',
        'n_gpus': 0,
        'logging_step': 300,
        'ema_start_step': 500,
        'ema_start': False,
        'seed': 2022
    }

if not torch.cuda.is_available():
    config['device'] = 'cpu'
else:
    config['n_gpus'] = torch.cuda.device_count()
    config['batch_size'] *= config['n_gpus']

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed

seed_everything(config['seed'])

2022

In [14]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(config['model_path'])

In [15]:
def build_bert_inputs(inputs, label, sentence_a, sentence_b, tokenizer):
  inputs_dict = tokenizer.encode_plus(sentence_a, sentence_b, add_special_tokens = True, return_token_type_ids =True, return_attention_mask = True)

  inputs['input_ids'].append(inputs_dict['input_ids'])
  inputs['token_type_ids'].append(inputs_dict['token_type_ids'])
  inputs['attention_mask'].append(inputs_dict['attention_mask'])
  inputs['labels'].append(label)

## 对偶数据增强

**a-b对，变成b-a对, 把两个句子换顺序**\
**我们的无监督数据增强就是用的对偶数据增强**\
**BERT输入 a，b两个句子，现在输入以b,a作为输入，增强样本**

In [16]:
import pandas as pd
from tqdm import tqdm
import json
def parse_data(path, data_type='train'):
  sentence_a = []
  sentence_b = []
  labels = []

  with open(path, 'r', encoding = 'utf8') as f:
    for line in tqdm(f.readlines(), desc=f'Reading {data_type} data'):
      line = json.loads(line)
      sentence_a.append(line['sentence1'])
      sentence_b.append(line['sentence2'])
      if data_type != 'test':
        labels.append(int(line['label']))
      else:
        labels.append(0)

  df = pd.DataFrame(zip(sentence_a, sentence_b, labels), columns = ['text_a', 'text_b', 'labels'])
  return df

In [17]:
# 无监督BERT输入
def build_unsup_bert_inputs(inputs, label, sentence_a, sentence_b, tokenizer):
  # 左右
  lr_inputs_dict = tokenizer.encode_plus(sentence_a, sentence_b, add_special_tokens = True, return_token_type_ids = True, return_attention_mask = True)
  # 右左
  rl_inputs_dict = tokenizer.encode_plus(sentence_b, sentence_a, add_special_tokens = True, return_token_type_ids = True, return_attention_mask = True)

  # 元组的形式
  inputs['input_ids'].append((lr_inputs_dict['input_ids'], rl_inputs_dict['input_ids']))
  inputs['token_type_ids'].append((lr_inputs_dict['token_type_ids'], rl_inputs_dict['token_type_ids']))
  inputs['attention_mask'].append((lr_inputs_dict['attention_mask'], rl_inputs_dict['attention_mask']))
  inputs['labels'].append(label)

## startswith()方法
startswith() 方法用于检查字符串是否是以指定子字符串开头\
如果是则返回 True，否则返回 False。如果参数 beg 和 end 指定值，则在指定范围内检查。\
str.startswith(str, beg=0,end=len(string));\
*参数*
```
str --检测的字符串。
strbeg --可选参数用于设置字符串检测的起始位置。
strend --可选参数用于设置字符串检测的结束位置。
```

In [18]:
from collections import defaultdict
def read_data(config, tokenizer):
  train_df = parse_data(config['train_file_path'], data_type = 'train')
  dev_df = parse_data(config['dev_file_path'], data_type = 'dev')
  test_df = parse_data(config['test_file_path'], data_type = 'test')

  data_df = {'train': train_df, 'dev': dev_df, 'test': test_df}
  processed_data = {}
  unsup_data = defaultdict(list)
  for data_type, df in data_df.items():
    inputs = defaultdict(list)
    if data_type == 'train':
      reversed_inputs = defaultdict(list)

    for i, row in tqdm(df.iterrows(), desc=f'Preprocessing {data_type} data', total = len(df)):
      label = 0 if data_type == 'test' else row[2]
      sentence_a, sentence_b = row[0], row[1]
      build_bert_inputs(inputs, label, sentence_a, sentence_b, tokenizer)

      if data_type.startswith('test'):
        build_bert_inputs(inputs, label, sentence_b, sentence_a, tokenizer)


      build_unsup_bert_inputs(unsup_data, label, sentence_a, sentence_b, tokenizer)

    processed_data[data_type] = inputs
  processed_data['unsup_data'] = unsup_data
  return processed_data

# processed_data
# {
#    'train':,
#    'dev':,
#    'test':,
#    'unsup_data':   # 数据量最大的
# }

In [19]:
data = read_data(config, tokenizer)

Preprocessing test data: 100%|██████████| 3861/3861 [00:06<00:00, 575.85it/s]
